# Quantum Simulation of Bell States with Noise Models and Interactive Visualization

This notebook simulates a two-qubit quantum system, primarily a Bell state, under various noise models like phase noise, amplitude damping, depolarizing noise, bit flip, and phase flip. It computes probabilities, fidelity, and entanglement, visualizes results with plots and Bloch spheres, and allows interactive exploration via widgets.

By: **Akhilesh Pant** (MCA)

**Amrapali University**, Haldwani

In [6]:
# ===============================================
# Quantum Simulation – Fully Interactive Notebook
# ===============================================

# -------------------------------
# 1. Import Libraries
# -------------------------------
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from qutip import (basis, tensor, ket2dm, fidelity, concurrence,
                   destroy, sigmax, sigmaz, qeye, mesolve, Bloch)
import ipywidgets as widgets
from IPython.display import display, clear_output

# -------------------------------
# 2. Qubit Definitions
# -------------------------------
def create_single_qubit(state="plus"):
    if state == "zero":
        return basis(2,0)
    elif state == "one":
        return basis(2,1)
    elif state == "plus":
        return (basis(2,0) + basis(2,1)).unit()
    elif state == "minus":
        return (basis(2,0) - basis(2,1)).unit()

def create_bell_pair():
    q0 = basis(2,0)
    q1 = basis(2,1)
    return (tensor(q0,q0) + tensor(q1,q1)).unit()

# -------------------------------
# 3. Noise Models
# -------------------------------
def apply_phase_noise(rho, gamma=0.1, tlist=[0,1]):
    sz1 = tensor(sigmaz(), qeye(2))
    sz2 = tensor(qeye(2), sigmaz())
    L = [gamma*sz1, gamma*sz2]
    result = mesolve(H=0*rho, rho0=rho, tlist=tlist, c_ops=L)
    return result.states[-1]

def apply_amplitude_damping(rho, gamma=0.1, tlist=[0,1]):
    c_ops = [gamma*tensor(destroy(2), qeye(2)), gamma*tensor(qeye(2), destroy(2))]
    result = mesolve(H=0*rho, rho0=rho, tlist=tlist, c_ops=c_ops)
    return result.states[-1]

def apply_depolarizing_noise(rho, p=0.1):
    I = qeye(2)
    return (1-p)*rho + (p/4)*tensor(I,I)

def apply_bit_flip(rho, p=0.1):
    X = sigmax()
    return (1-p)*rho + p*tensor(X,X)*rho*tensor(X,X)

def apply_phase_flip(rho, p=0.1):
    Z = sigmaz()
    return (1-p)*rho + p*tensor(Z,Z)*rho*tensor(Z,Z)

# -------------------------------
# 4. Measurement Functions
# -------------------------------
def compute_probabilities(rho):
    rho_q0 = rho.ptrace(0)
    rho_q1 = rho.ptrace(1)
    return {
        "q0_0": (rho_q0*rho_q0.dag()).diag()[0].real,
        "q0_1": (rho_q0*rho_q0.dag()).diag()[1].real,
        "q1_0": (rho_q1*rho_q1.dag()).diag()[0].real,
        "q1_1": (rho_q1*rho_q1.dag()).diag()[1].real
    }

def compute_fidelity(rho, rho_target):
    return fidelity(rho, rho_target)

def compute_entanglement(rho):
    return concurrence(rho)

# -------------------------------
# 5. Simulation Function
# -------------------------------
def run_simulation(noise_type, gamma, p, t_steps, runs):
    bell_state = create_bell_pair()
    bell_density = ket2dm(bell_state)
    tlist = np.linspace(0, t_steps, max(t_steps, 2))

    prob_over_time = []
    fidelity_over_time = []
    entanglement_over_time = []

    for t in tlist:
        if noise_type == "Phase Noise":
            rho = apply_phase_noise(bell_density, gamma=gamma, tlist=[0,t])
        elif noise_type == "Amplitude Damping":
            rho = apply_amplitude_damping(bell_density, gamma=gamma, tlist=[0,t])
        elif noise_type == "Depolarizing Noise":
            rho = apply_depolarizing_noise(bell_density, p=p)
        elif noise_type == "Bit Flip":
            rho = apply_bit_flip(bell_density, p=p)
        elif noise_type == "Phase Flip":
            rho = apply_phase_flip(bell_density, p=p)
        else:
            rho = bell_density

        prob_over_time.append(compute_probabilities(rho))
        fidelity_over_time.append(compute_fidelity(rho, bell_density))
        entanglement_over_time.append(compute_entanglement(rho))

    return tlist, prob_over_time, fidelity_over_time, entanglement_over_time

# -------------------------------
# 6. Plot Functions
# -------------------------------
def plot_probabilities(prob_dict):
    plt.figure(figsize=(6,4))
    plt.bar(prob_dict.keys(), prob_dict.values(), color='skyblue')
    plt.ylabel("Probability")
    plt.title("Final Qubit Probabilities")
    plt.show()

def plot_fidelity_time(tlist, fidelity_values):
    plt.figure(figsize=(6,4))
    plt.plot(tlist, fidelity_values, '-o', color='green')
    plt.xlabel("Time Steps")
    plt.ylabel("Fidelity")
    plt.title("Fidelity vs Time")
    plt.grid(True)
    plt.show()

def plot_entanglement_time(tlist, ent_values):
    plt.figure(figsize=(6,4))
    plt.plot(tlist, ent_values, '-o', color='orange')
    plt.xlabel("Time Steps")
    plt.ylabel("Concurrence")
    plt.title("Entanglement vs Time")
    plt.grid(True)
    plt.show()

def show_bloch_sphere_3d():
    b = Bloch()
    b.add_states([create_single_qubit("plus"), create_single_qubit("minus")])
    b.render()
    plt.show()

def plot_3d_probability_surface(tlist, prob_time):
    fig = plt.figure(figsize=(8,6))
    ax = fig.add_subplot(111, projection='3d')

    x = np.array(tlist)
    y_labels = ['q0_0','q0_1','q1_0','q1_1']
    y = np.arange(len(y_labels))

    for idx, label in enumerate(y_labels):
        z = [p[label] for p in prob_time]
        ax.plot(x, np.full_like(x, y[idx]), z, label=label)

    ax.set_xlabel('Time Steps')
    ax.set_ylabel('States')
    ax.set_yticks(y)
    ax.set_yticklabels(y_labels)
    ax.set_zlabel('Probability')
    ax.set_title('3D Probability Evolution')
    ax.legend()
    plt.show()

# -------------------------------
# 7. Widgets Interface
# -------------------------------
noise_dropdown = widgets.Dropdown(
    options=['None', 'Phase Noise', 'Amplitude Damping', 'Depolarizing Noise', 'Bit Flip', 'Phase Flip'],
    value='Phase Noise',
    description='Noise Type:',
)

gamma_slider = widgets.FloatSlider(value=0.05, min=0.0, max=1.0, step=0.01, description='Gamma:')

p_slider = widgets.FloatSlider(value=0.05, min=0.0, max=1.0, step=0.01, description='Probability p:')

time_steps_slider = widgets.IntSlider(value=50, min=2, max=200, step=1, description='Time Steps:')

run_button = widgets.Button(description="Run Simulation", button_style='success')
output = widgets.Output()

def on_button_clicked(b):
    with output:
        clear_output(wait=True)
        noise_type = noise_dropdown.value
        gamma = gamma_slider.value
        p = p_slider.value
        t_steps = time_steps_slider.value

        tlist, prob_time, fidelity_time, ent_time = run_simulation(noise_type, gamma, p, t_steps, 1)

        final_prob = prob_time[-1]

        print(f"Simulation Results for {noise_type}:")
        print(f"Final Qubit Probabilities: {final_prob}")
        print(f"Final Fidelity: {fidelity_time[-1]:.4f}")
        print(f"Final Entanglement: {ent_time[-1]:.4f}")

        plot_probabilities(final_prob)
        plot_fidelity_time(tlist, fidelity_time)
        plot_entanglement_time(tlist, ent_time)
        plot_3d_probability_surface(tlist, prob_time)
        show_bloch_sphere_3d()

run_button.on_click(on_button_clicked)

display(noise_dropdown, gamma_slider, p_slider, time_steps_slider, run_button, output)

Dropdown(description='Noise Type:', index=1, options=('None', 'Phase Noise', 'Amplitude Damping', 'Depolarizin…

FloatSlider(value=0.05, description='Gamma:', max=1.0, step=0.01)

FloatSlider(value=0.05, description='Probability p:', max=1.0, step=0.01)

IntSlider(value=50, description='Time Steps:', max=200, min=2)

Button(button_style='success', description='Run Simulation', style=ButtonStyle())

Output()

In [10]:
# ===============================================
# Quantum Simulation – Fully Interactive Notebook with Saving
# ===============================================

# -------------------------------
# 1. Import Libraries
# -------------------------------
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from qutip import (basis, tensor, ket2dm, fidelity, concurrence,
                   destroy, sigmax, sigmaz, qeye, mesolve, Bloch)
import ipywidgets as widgets
from IPython.display import display, clear_output
import pickle
import os

# -------------------------------
# 2. Qubit Definitions
# -------------------------------
def create_single_qubit(state="plus"):
    """Create a single qubit in a specified state."""
    if state == "zero":
        return basis(2,0)
    elif state == "one":
        return basis(2,1)
    elif state == "plus":
        return (basis(2,0) + basis(2,1)).unit()
    elif state == "minus":
        return (basis(2,0) - basis(2,1)).unit()

def create_bell_pair():
    """Create a Bell pair (entangled state)."""
    q0 = basis(2,0)
    q1 = basis(2,1)
    return (tensor(q0,q0) + tensor(q1,q1)).unit()

# -------------------------------
# 3. Noise Models
# -------------------------------
def apply_phase_noise(rho, gamma=0.1, tlist=[0,1]):
    sz1 = tensor(sigmaz(), qeye(2))
    sz2 = tensor(qeye(2), sigmaz())
    L = [gamma*sz1, gamma*sz2]
    result = mesolve(H=0*rho, rho0=rho, tlist=tlist, c_ops=L)
    return result.states[-1]

def apply_amplitude_damping(rho, gamma=0.1, tlist=[0,1]):
    c_ops = [gamma*tensor(destroy(2), qeye(2)), gamma*tensor(qeye(2), destroy(2))]
    result = mesolve(H=0*rho, rho0=rho, tlist=tlist, c_ops=c_ops)
    return result.states[-1]

def apply_depolarizing_noise(rho, p=0.1):
    I = qeye(2)
    return (1-p)*rho + (p/4)*tensor(I,I)

def apply_bit_flip(rho, p=0.1):
    X = sigmax()
    return (1-p)*rho + p*tensor(X,X)*rho*tensor(X,X)

def apply_phase_flip(rho, p=0.1):
    Z = sigmaz()
    return (1-p)*rho + p*tensor(Z,Z)*rho*tensor(Z,Z)

# -------------------------------
# 4. Measurement Functions
# -------------------------------
def compute_probabilities(rho):
    """Compute probabilities of qubits being 0 or 1."""
    rho_q0 = rho.ptrace(0)
    rho_q1 = rho.ptrace(1)
    return {
        "q0_0": (rho_q0*rho_q0.dag()).diag()[0].real,
        "q0_1": (rho_q0*rho_q0.dag()).diag()[1].real,
        "q1_0": (rho_q1*rho_q1.dag()).diag()[0].real,
        "q1_1": (rho_q1*rho_q1.dag()).diag()[1].real
    }

def compute_fidelity(rho, rho_target):
    return fidelity(rho, rho_target)

def compute_entanglement(rho):
    return concurrence(rho)

# -------------------------------
# 5. Simulation Function
# -------------------------------
def run_simulation(noise_type, gamma, p, t_steps, runs):
    bell_state = create_bell_pair()
    bell_density = ket2dm(bell_state)
    tlist = np.linspace(0, t_steps, max(t_steps, 2))

    prob_over_time = []
    fidelity_over_time = []
    entanglement_over_time = []

    for t in tlist:
        if noise_type == "Phase Noise":
            rho = apply_phase_noise(bell_density, gamma=gamma, tlist=[0,t])
        elif noise_type == "Amplitude Damping":
            rho = apply_amplitude_damping(bell_density, gamma=gamma, tlist=[0,t])
        elif noise_type == "Depolarizing Noise":
            rho = apply_depolarizing_noise(bell_density, p=p)
        elif noise_type == "Bit Flip":
            rho = apply_bit_flip(bell_density, p=p)
        elif noise_type == "Phase Flip":
            rho = apply_phase_flip(bell_density, p=p)
        else:
            rho = bell_density

        prob_over_time.append(compute_probabilities(rho))
        fidelity_over_time.append(compute_fidelity(rho, bell_density))
        entanglement_over_time.append(compute_entanglement(rho))

    return tlist, prob_over_time, fidelity_over_time, entanglement_over_time

# -------------------------------
# 6. Plot Functions
# -------------------------------
def plot_probabilities(prob_dict):
    plt.figure(figsize=(6,4))
    plt.bar(prob_dict.keys(), prob_dict.values(), color='skyblue')
    plt.ylabel("Probability")
    plt.title("Final Qubit Probabilities")
    plt.show()

def plot_fidelity_time(tlist, fidelity_values):
    plt.figure(figsize=(6,4))
    plt.plot(tlist, fidelity_values, '-o', color='green')
    plt.xlabel("Time Steps")
    plt.ylabel("Fidelity")
    plt.title("Fidelity vs Time")
    plt.grid(True)
    plt.show()

def plot_entanglement_time(tlist, ent_values):
    plt.figure(figsize=(6,4))
    plt.plot(tlist, ent_values, '-o', color='orange')
    plt.xlabel("Time Steps")
    plt.ylabel("Concurrence")
    plt.title("Entanglement vs Time")
    plt.grid(True)
    plt.show()

def show_bloch_sphere_3d():
    b = Bloch()
    b.add_states([create_single_qubit("plus"), create_single_qubit("minus")])
    b.render()
    plt.show()

def plot_3d_probability_surface(tlist, prob_time):
    fig = plt.figure(figsize=(8,6))
    ax = fig.add_subplot(111, projection='3d')

    x = np.array(tlist)
    y_labels = ['q0_0','q0_1','q1_0','q1_1']
    y = np.arange(len(y_labels))

    for idx, label in enumerate(y_labels):
        z = [p[label] for p in prob_time]
        ax.plot(x, np.full_like(x, y[idx]), z, label=label)

    ax.set_xlabel('Time Steps')
    ax.set_ylabel('States')
    ax.set_yticks(y)
    ax.set_yticklabels(y_labels)
    ax.set_zlabel('Probability')
    ax.set_title('3D Probability Evolution')
    ax.legend()
    plt.show()

# -------------------------------
# 7. Save and Load Simulation
# -------------------------------
def save_simulation_data(filename, data):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)
    print(f"Simulation saved to {filename}")

def load_simulation_data(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

# -------------------------------
# 8. Widgets Interface
# -------------------------------
noise_dropdown = widgets.Dropdown(
    options=['None', 'Phase Noise', 'Amplitude Damping', 'Depolarizing Noise', 'Bit Flip', 'Phase Flip'],
    value='Phase Noise',
    description='Noise Type:',
)

gamma_slider = widgets.FloatSlider(value=0.05, min=0.0, max=1.0, step=0.01, description='Gamma:')
p_slider = widgets.FloatSlider(value=0.05, min=0.0, max=1.0, step=0.01, description='Probability p:')
time_steps_slider = widgets.IntSlider(value=50, min=2, max=200, step=1, description='Time Steps:')

run_button = widgets.Button(description="Run Simulation", button_style='success')
output = widgets.Output()

# -------------------------------
# 9. Button Callback
# -------------------------------
def on_button_clicked(b):
    with output:
        clear_output(wait=True)
        noise_type = noise_dropdown.value
        gamma = gamma_slider.value
        p = p_slider.value
        t_steps = time_steps_slider.value

        # Run the simulation
        tlist, prob_time, fidelity_time, ent_time = run_simulation(noise_type, gamma, p, t_steps, 1)

        final_prob = prob_time[-1]

        # Prepare data for saving
        simulation_data = {
            "noise_type": noise_type,
            "gamma": gamma,
            "p": p,
            "tlist": tlist,
            "prob_time": prob_time,
            "fidelity_time": fidelity_time,
            "ent_time": ent_time,
            "final_prob": final_prob
        }

        # Save to a .pkl file
        filename = f"quantum_simulation_{noise_type.replace(' ', '_')}.pkl"
        save_simulation_data(filename, simulation_data)

        # Display results
        print(f"Simulation Results for {noise_type}:")
        print(f"Final Qubit Probabilities: {final_prob}")
        print(f"Final Fidelity: {fidelity_time[-1]:.4f}")
        print(f"Final Entanglement: {ent_time[-1]:.4f}")

        # Plot visualizations
        plot_probabilities(final_prob)
        plot_fidelity_time(tlist, fidelity_time)
        plot_entanglement_time(tlist, ent_time)
        plot_3d_probability_surface(tlist, prob_time)
        show_bloch_sphere_3d()

# Attach button action
run_button.on_click(on_button_clicked)

# Display widgets
display(noise_dropdown, gamma_slider, p_slider, time_steps_slider, run_button, output)


Dropdown(description='Noise Type:', index=1, options=('None', 'Phase Noise', 'Amplitude Damping', 'Depolarizin…

FloatSlider(value=0.05, description='Gamma:', max=1.0, step=0.01)

FloatSlider(value=0.05, description='Probability p:', max=1.0, step=0.01)

IntSlider(value=50, description='Time Steps:', max=200, min=2)

Button(button_style='success', description='Run Simulation', style=ButtonStyle())

Output()